In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas import json_normalize
import folium

print('Libraries imported.')

Libraries imported.


In [4]:
CLIENT_ID = 'DYOJFWB2RDCT2XVXBQLND5FIOD1S1VJ2QBKXT2ADJ5UBFCCN'
CLIENT_SECRET = 'WYK0FEPFXEEWK0MLGFL2T1EYPQZ3PPR0FSUQJAQI0SFLRRPJ'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: DYOJFWB2RDCT2XVXBQLND5FIOD1S1VJ2QBKXT2ADJ5UBFCCN


In [5]:
LIMIT = 100
radius = 500
cities = ["İstanbul, Türkiye", 'Ankara, Türkiye', 'İzmir, Türkiye', 'Antalya, Türkiye', 'Adana, Türkiye']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        radius,
        LIMIT,
        "5283c7b4e4b094cb91ec88d7") #kebab restaurants in Turkey
    results[city] = requests.get(url).json()

In [6]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [7]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of kebab places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of kebab places in İstanbul, Türkiye =  76
Showing Top 100
Total number of kebab places in Ankara, Türkiye =  51
Showing Top 100
Total number of kebab places in İzmir, Türkiye =  39
Showing Top 100
Total number of kebab places in Antalya, Türkiye =  7
Showing Top 100
Total number of kebab places in Adana, Türkiye =  30
Showing Top 100


In [8]:
maps[cities[0]]

In [9]:
maps[cities[1]]

In [10]:
maps[cities[2]]

In [11]:
maps[cities[3]]

In [12]:
maps[cities[4]]

In [13]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

İstanbul, Türkiye
Mean Distance from Mean coordinates
0.0032184533039484983
Ankara, Türkiye
Mean Distance from Mean coordinates
0.003514018473895761
İzmir, Türkiye
Mean Distance from Mean coordinates
0.00749220948518137
Antalya, Türkiye
Mean Distance from Mean coordinates
0.0016060012994064557
Adana, Türkiye
Mean Distance from Mean coordinates
0.0033505829825406556


In [14]:
maps[cities[0]]

In [15]:
maps[cities[1]]

In [16]:
maps[cities[2]]

In [17]:
maps[cities[3]]

In [18]:
maps[cities[4]]

In [20]:
city = 'İzmir, Türkiye'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))

İzmir, Türkiye
Mean Distance from Mean coordinates
0.007156594277636101
